In [1]:
from math import sqrt
from statistics import mean

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as pp
import itertools
import copy
from functools import reduce

/Users/samueleceol/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [114]:
#df = pd.read_csv("CRISP/CSV/db/farming_system.csv")
#df = pd.read_csv("CRISP/CSV/db/fs_landscapes.csv")
#df = pd.read_csv("CRISP/CSV/db/commodities.csv")
df = pd.read_csv("CRISP/CSV/db/fs_factors.csv")
df

,ID,Label,FS name,FS macro region,Type-1,Description
0,1,Increase in average temperatures,Lowland Rice,EAP,Hazard,Increase in Temperature of 0.02-0.03 °C per ye...
1,2,Rising sea levels,Lowland Rice,EAP,Hazard,The tides are rising at a rate close to ~2 cm/...
2,3,Increase in drought events,Lowland Rice,EAP,Hazard,The risk of drought often occurs during the Wi...
3,4,Increase in average precipitation,Lowland Rice,EAP,Hazard,"In recent years, farmers in many provinces hav..."
4,5,Decrease in average precipitation,Lowland Rice,EAP,Hazard,"The temperature has increased, annual rainfall..."
...,...,...,...,...,...,...
1733,1734,Farmers engaged in agri system,Root Crop,SSA,Exposure,The root and tuber crop farming system d suppo...
1734,1735,Area of land used,Root Crop,SSA,Exposure,The root and tuber crop farming system occupie...
1735,1736,Risk to farmers of chronic poverty due to Clim...,Root Crop,SSA,Risk,Poverty is relatively high with about half the...
1736,1737,Risk to farmers of food insecurity due to Clim...,Root Crop,SSA,Risk,Cassava alone supplies more food calories than...


In [115]:
def PLI_single(df, col_index):
    res = {}
    observed_vals = []

    for index, value in enumerate(df.iloc[:, col_index]): # Iterate through the values in the column
        if value in observed_vals: res[observed_vals.index(value)].append(index)
        else: 
            observed_vals.append(value) 
            res[len(observed_vals) - 1] = [index] # Add index to dictionary

    return {k: v for k, v in res.items() if len(v) > 1}

def PLI_intersect(p1, p2):
    hash = {}
    for group in p1:
        for row in p1[group]:
            hash[row] = group
    
    pli = {}
    for group in p2:
        for row in p2[group]:
            if(row in hash):
                key = [hash[row], group]

                if(tuple(key) not in pli):
                    pli[tuple(key)] = [row]
                else:
                    pli[tuple(key)] = pli[tuple(key)] + [row]
    
    return {k: v for k, v in pli.items() if len(v) > 1}

def multi_column_PLI_intersect(df, col_idxs):
    pli = PLI_single(df, col_idxs[0])

    for i in range(1, len(col_idxs)):
        pli = PLI_intersect(pli, PLI_single(df, col_idxs[i]))

    return pli

def key_error(pli):
    key_error = 0
    for key in pli:
        key_error += len(pli[key]) - 1
    return key_error

In [116]:
def UCC_discovery(df):
    k = 0
    E = []
    F = []
    E.insert(k, [[[col_idx], PLI_single(df, col_idx)] for col_idx in range(0, len(df.columns))])
    F.insert(k, UCC_prune(E[k]))

    while len(F[k]) != 0:
        E.insert(k+1, UCC_candidates(F[k]))
        F.insert(k+1, UCC_prune(E[k+1]))
        k += 1

    uccs = []
    for j in range(0, k):
        ucc_level = [e[0] for e in E[j] if e not in F[j]]
        for col_comb in ucc_level:
            uccs.append([list(df.columns)[idx] for idx in col_comb])
    return uccs

def UCC_prune(E):
    F = []
    for e in E:
        if e[1]: F.append([e[0], e[1]])
    return F

def UCC_candidates(F):
    E = []
    for f_1, p_1 in F:
        for f_2, p_2 in F:
            # Column combination differs only by one and
            # Last column idx for f_1 < Last column idx for f_2
            if((f_1[:-1] == f_2[:-1]) and (f_1[-1] < f_2[-1])):
                f = f_1 + f_2[-1:]
                
                # Check subsets of f of size len(f) - 1
                if(all(item in list(zip(*F))[0] for item in [list(t) for t in itertools.combinations(f, len(f)-1)])):
                    p = PLI_intersect(p_1, p_2)
                    E.append([f, p])
    return E


In [117]:
UCC_discovery(df)

[['ID']]

In [118]:
def FD_discovery(df):
    k = 0
    E = []
    C = []
    F = []

    primary_keys = [] #TODO: Remove
 
    E.insert(k, [[[col_idx], PLI_single(df, col_idx)] for col_idx in range(0, len(df.columns))])
    C.insert(k, {str(tuple([col_idx])): list(range(0,len(df.columns))) for col_idx in range(0, len(df.columns))})
    F.insert(k, FD_prune(E[k], C[k], primary_keys))
    while F[k]:
        E.insert(k+1, FD_candidates(F[k]))
        C.insert(k+1, FD_dependencies(E[k+1], C[k], df))
        F.insert(k+1, FD_prune(E[k+1], C[k+1], primary_keys))
        k = k+1

def FD_prune(E, C_plus, primary_keys):
    F = copy.deepcopy(E)
    for e in E:
        X = e[0]
        p = e[1]

        if(not C_plus[str(tuple(X))]): 
            F.remove([X, p])
        if(not p): # Check superkey
            primary_keys.append(X) # TODO: Remove
            for A in [c for c in C_plus[str(tuple(X))] if c not in X]:
                aug_lhs_list = []
                for B in X: aug_lhs_list.append(sorted([x for x in X if x != B] + [A]))
                
                # FIX for early pruning of primary keys - TODO: Remove
                to_be_removed = []
                for subset in aug_lhs_list: 
                    for pk in primary_keys:
                        if all(x in subset for x in pk):
                            to_be_removed.append(subset)
                aug_lhs_list = [subset for subset in aug_lhs_list if subset not in to_be_removed]
                # END FIX

                if(all(str(tuple(aug_lhs)) in C_plus for aug_lhs in aug_lhs_list)):
                    if(all(A in C_plus[str(tuple(aug_lhs))] for aug_lhs in aug_lhs_list)): 
                        print(f"{X} -> {A}")
            F.remove([X, p])
    return F

def FD_candidates(F):
    E = []
    for f_1, p_1 in F:
        for f_2, p_2 in F:
            # Column combination differs only by one and
            # Last column idx for f_1 < Last column idx for f_2
            if((f_1[:-1] == f_2[:-1]) and (f_1[-1] < f_2[-1])):
                f = f_1 + f_2[-1:]
                
                # Check subsets of f of size len(f) - 1
                if(all(item in list(zip(*F))[0] for item in [list(t) for t in itertools.combinations(f, len(f)-1)])):
                    p = PLI_intersect(p_2, p_1)
                    E.append([f, p])
    return E

def FD_dependencies(E, C, df):
    C_plus = {}
        
    for X, p in E:
        prev_rhs = [C[str(tuple([x for x in X if x != B]))] for B in X]
        C_plus[str(tuple(X))] = [x for x in prev_rhs[0] if all(x in subset for subset in prev_rhs)] # TODO: Understand this step

    for X, p in E:
        for A in [x for x in X if x in C_plus[str(tuple(X))]]:
            subset = [x for x in X if x != A]
            subset_error = key_error(multi_column_PLI_intersect(df, subset))
            
            if(subset_error == key_error(p)): 
                print(f"{subset} -> {A}")
                C_plus[str(tuple(X))] = list(set(C_plus[str(tuple(X))]) - set([A]))

                for B in set(range(0, len(df.columns))) - set(X):
                    C_plus[str(tuple(X))] = list(set(C_plus[str(tuple(X))]) - set([B]))
    
    return C_plus

FD_discovery(df)

[0] -> 1
[0] -> 2
[0] -> 3
[0] -> 4
[0] -> 5
[1, 2] -> 4


In [ ]:
import argparse
import pandas as pd
from functools import reduce

def get_pli_from(col_index, r):
    column = r.iloc[:,col_index]
    pli = {} # valore: [indexes]
    known_elements = []
    for index, value in enumerate(column):
        if value in known_elements:
            pli.get(known_elements.index(value)).append(index)
        else:
            pli.update({len(known_elements): [index]})
            known_elements.append(value)

    pli = dict(filter(lambda x: len(x[1]) > 1, pli.items()))
    return pli

def get_intersected_row_pairs(first_pli, second_pli):
    prob_table = {}
    for key, value in first_pli.items():
        for v in value:
            prob_table[v] = key

    row_pairs = {}
    for key, value in second_pli.items():
        for v in value:
            if v in prob_table:
                pair = ", ".join([str(key), str(prob_table[v])])
                if pair in row_pairs:
                    row_pairs.get(pair).append(v)
                else:
                    row_pairs.update({pair: [v]})

    row_pairs = dict(filter(lambda x: len(x[1]) > 1, row_pairs.items()))
    return row_pairs

def get_pli_from_indexes(col_indexes, r):
    pli = get_pli_from(col_indexes[0], r)
    for col_index in col_indexes:
        if col_index == col_indexes[0]:
            continue
        pli = get_intersected_row_pairs(pli, get_pli_from(col_index, r))
    return pli

def get_key_error_from_pli(pli):
    count = 0
    for group in pli.items():
        count += (len(group[1]) - 1)
    return count

def apriori(r):
    k = 0
    E = []
    C = []
    F = []

    primary_keys = []
    
    E.append([[[i], get_pli_from(i, r)] for i in range(len(r.columns))])
    C.append({str(i): list(range(len(r.columns))) for i in range(len(r.columns))})
    F.insert(k, prune(E[k], C[k], r, primary_keys))

    while (F[k]):
        #print(f"Working on level: {k + 1}")
        E.insert(k+1, candidates(F[k]))
        C.insert(k+1, dependencies(E[k+1], C[k], r))
        F.insert(k+1, prune(E[k+1], C[k+1], r, primary_keys))
        k += 1

def candidates(F):
    E = []
    for f1, p1 in F:
        for f2, p2 in F:
            if f1[:-1] == f2[:-1] and f2[-1] > f1[-1]:
                f = f1 + [f2[-1]]
                p = get_intersected_row_pairs(p1, p2)

                should_append = True
                for f3 in f:
                    f_subset = [x for x in f if x != f3]
                    if f_subset not in list(map(lambda x: x[0], F)):
                        should_append = False
                        break

                if should_append:
                    E.append([f, p])

    return E

def dependencies(E, C, r):
    C_plus = {}
    for e in E:
        X = e[0]
        pli = e[1]
        
        subsets = []
        for x in X:
            subsets.append([element for element in X if element != x])

        subsets_rhs_list = list(map(lambda subset: C["".join([str(x) for x in subset])], subsets))

        C_plus["".join([str(x) for x in X])] = [x for x in subsets_rhs_list[0] if all(x in subset for subset in subsets_rhs_list)]

    for e in E:
        X = e[0]
        pli = e[1]
        key_error_pli = get_key_error_from_pli(pli)
        for A in [x for x in X if x in C_plus["".join([str(x) for x in X])]]:
            plis = get_pli_from_indexes([x for x in X if x != A], r)
            key_error_plis = get_key_error_from_pli(plis)

            if key_error_pli == key_error_plis:
                print(f"minimal FD: {[x for x in X if x != A]} -> {A}")
                C_plus["".join([str(x) for x in X])] = [x for x in C_plus["".join([str(x) for x in X])] if x != A]

                for B in [x for x in list(range(len(r.columns))) if x not in X]:
                    C_plus["".join([str(x) for x in X])] = [x for x in C_plus["".join([str(x) for x in X])] if x != B]

    return C_plus

def prune(E, C, r, primary_keys):
    F = E

    for e in E:
        X = e[0]
        pli = e[1]

        if not C["".join([str(x) for x in X])]:
            F = list(filter(lambda x: not (x[0] == X and x[1] == pli), F))
        if not get_pli_from_indexes(X, r):
            #print(f"{X} is a key")
            primary_keys.append(X)
            for A in [x for x in C["".join([str(x) for x in X])] if x not in X]:
                subsets = []
                for x in X:
                    subsets.append(sorted([element for element in X if element != x] + [A]))
                
                # FIX for early pruning of primary keys
                to_be_removed = []
                for subset in subsets: 
                    for pk in primary_keys:
                        if all(x in subset for x in pk):
                            to_be_removed.append(subset)
                subsets = [subset for subset in subsets if subset not in to_be_removed]
                # END FIX
                
                if all("".join([str(x) for x in subset]) in C for subset in subsets):
                    if all(A in C["".join([str(x) for x in subset])] for subset in subsets):
                        print(f"minimal FD: {X} -> {A}")
            F = list(filter(lambda x: not (x[0] == X and x[1] == pli), F))
    return F

apriori(df)